In [1]:
import os
import gym
import time
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import deque
from gym.envs.registration import register

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### 미끄러지지 않는 환경을 세팅합니다.

In [2]:
register(
    id='LakeEnv-v1',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '4x4', 'is_slippery': False}
)
env = gym.make('LakeEnv-v1')

### 해당 state에 대한 정보를 받아오기 위한 과정

In [3]:
# target 임베딩, input shape -> (1, 16)
# state를 preprocessed sequenced, state를 우리가 원하는 형태로 바꾸는 함수
def one_hot(x):
    return np.identity(16)[x:x + 1].astype(np.float32)

In [4]:
# Set DQN parameters
num_episodes = 20000
learning_rate = 0.1 
dis = .99     # discount_reward
h_size = 10    # 은닉층의 shape -> input(1,16) * hidden1(16,10) -> hidden2(10,4) -> output(1,4)

REPLAY_MEMORY = 2500   # 버퍼의 최대 크기

# buffer parameters
random_sample = 10    # 버퍼가 다 찬 후 num_episodes에서 언제 random.sample을 할지 결정하는 파라미터 
num_sample = 50   #sample을 뽑는 것을 얼마나 반복할 것인지 결정하는 파라미터
num_choice = 10   # buffer에서 몇 개를 추출할 것인지 결정하는 파라미터

# Input and output size based on the Env
input_size = env.observation_space.n
output_size = env.action_space.n

# 업데이트되는 W를 통해 현재 state를 계산합니다.
# weight 1  (16, 10)
W1 = tf.Variable(tf.random.uniform([input_size, h_size], 0, 0.01), dtype=tf.float32)

# weight 2  (10, 4)
W2 = tf.Variable(tf.random.uniform([h_size, output_size], 0, 0.01), dtype=tf.float32)

# optimizer
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)

In [ ]:
start_time = time.time()
# rewards per episode
rList = []
buffer = deque(maxlen=REPLAY_MEMORY)
save_weight = []

for i in range(num_episodes):
    # Reset environment and get first new observation
    state = env.reset()
    rAll = 0
    step_count = 0
    done = False
    
    e = 1. / ((i / 50) + 10)
    # The Q-Table learning algorithm
    while not done:
        # Choose an action by greedly (with a chance of random action) from the Q-network
        dense_layer_1 = tf.matmul(one_hot(state), W1)
        activation_func_1 = tf.nn.relu(dense_layer_1)
        dense_layer_2 = tf.matmul(activation_func_1, W2)
        q_value = np.array(dense_layer_2.numpy())

        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_value)

        state_next, reward, done, _ = env.step(action)
        # gym ai가 action을 취하고 얻은 정보를 학습시키지않고 buffer에 담습니다. 
        buffer.append((state, action, reward, state_next, done))
        # 기존에 버퍼 크기를 지정
        state = state_next
        step_count += 1
        if step_count > 100:
          break

    # buffer를 통한 학습 구현
    # -----------------------------------------------------------------------------------------
    if len(buffer) < num_choice:
        continue
    if i % random_sample == 1:
        for _ in range(num_sample):
            # buffer에서 학습할 데이터를 랜덤하게 뽑아서 학습합니다.
            sample = random.sample(buffer, num_choice)

            # sample의 개수 많큼 for문을 돌립니다
            for state, action, reward, state_next, done in sample:
                x_stack = np.empty(0, dtype=np.float32).reshape(0, input_size)
                y_stack = np.empty(0, dtype=np.float32).reshape(0, output_size)
                # 타겟값 구하기
                if done:
                    q_value[0, action] = reward
                else:
                    next_dense_layer_1 = tf.matmul(one_hot(state_next), W1)
                    activation_func_1 = tf.nn.relu(next_dense_layer_1)

                    next_dense_layer_2 = tf.matmul(activation_func_1, W2)
                    q_score_next = np.array(next_dense_layer_2.numpy())

                    q_value[0, action] = reward + dis * np.max(q_score_next)
            
                # Qpred_dense_layer_1 = tf.matmul(one_hot(state), W1)
                # Qpred_activation_1 = tf.nn.relu(Qpred_dense_layer_1)
                # Qpred_dense_layer_2 = tf.matmul(Qpred_activation_1, W2)
                y_stack = np.vstack([y_stack, q_value])
                x_stack = np.vstack([x_stack, one_hot(state)])

            loss = lambda: tf.reduce_sum(input_tensor = (tf.square(y_stack - tf.matmul(tf.nn.relu(tf.matmul(x_stack, W1)), W2))))                              
                                            
            # optimizer, loss가 작아지는 방향으로 W 업데이트
            optimizer.minimize(loss, var_list=[W1, W2])
    # -----------------------------------------------------------------------------------------
    if i % 100 == 0:
        print(f'진행률: {round(i / num_episodes, 2) * 100}%')

### TEST

In [6]:
state = env.reset()
reward_sum = 0
while True:
    dense_layer_1 = tf.matmul(one_hot(state), W1)
    activation_func_1 = tf.nn.relu(dense_layer_1)
    dense_layer_2 = tf.matmul(activation_func_1, W2)
    q_value = np.array(dense_layer_2.numpy())
    
    action = np.argmax(q_value)
    next_state, reward, done, _ = env.step(action)
    reward_sum += reward
    if done:
        print(f"Total score: {reward_sum}")
        break

KeyboardInterrupt: ignored

제대로 학습이 이루어지지 않으면 길을 못찾는다.

# 결론

DQN 2013은 기존에 Q-Network가 가지고 있는 연속된 데이터간의 유사성으로 인한 학습 능력 저하 문제를 데이터를 바로 학습에 사용하지않고 buffer를 통해 랜덤으로 추출하는 방법을 사용해서 해결했습니다. 

하지만 DQN 2013에서는 타겟값과 예측값이 같은 가중치 (W1, W2)를 공유해서 성능이 좋지않았습니다.

## error 코드 문제점 및 수정사항
```python
for i in range(num_episodes):
    state = env.reset()
    rAll = 0
    done = False
    
    e = 1. / ((i / 50) + 10)

    while not done:
        dense_layer_1 = tf.matmul(one_hot(state), W1)
        activation_func = tf.nn.relu(dense_layer_1)

        dense_layer_2 = tf.matmul(activation_func, W2)
        activation_func = tf.nn.relu(dense_layer_2)

        q_value = np.array(activation_func.numpy())

        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_value)

        state_next, reward, done, _ = env.step(action)
        buffer.append((state, action, reward, state_next, done))
        state = state_next

        if len(buffer) > deque_maxlen:
            pass

        if num_episodes % random_sample == 1:
            for _ in range(num_sample):
                sample = random.sample(buffer, num_choice)

                for state, action, reward, state_next, done in sample:
                    if done:
                        q_value[0, action] = reward
                    else:
                        dense_layer_1 = tf.matmul(one_hot(state_next), W1)
                        activation_func = tf.nn.relu(dense_layer_1)

                        dense_layer_2 = tf.matmul(activation_func, W2)
                        activation_func = tf.nn.relu(dense_layer_2)
                        
                        q_score_next = np.array(activation_func.numpy())
                        q_value[0, action] = reward + dis * np.max(q_score_next)

                dense_layer_1 = tf.matmul(one_hot(state), W1)
                activation_func = tf.nn.relu(dense_layer_1)

                loss = lambda: tf.reduce_sum(input_tensor=tf.square(q_value - tf.nn.relu(tf.matmul(activation_func, W2))))
                optimizer.minimize(loss, var_list=[W1, W2])
```
<br>

1. 여기서 num_episode는 에피소드를 얼마나 진행할지 정하는 파라미터이고, 우리가 하고자 하는 것은 에피소드를 진행하면서 에피소드가 10이 될 때마다 랜덤해서 추출하는 것이기 때문에 num_episodes가 i로 바꿔야합니다.

2. while not done 루프를 통해 우리가 얻는 것은 버퍼에 들어가는 데이터고 우리는 바로 학습하는 것이 아니라 버퍼에서 데이터를 랜덤하게 추출해서 학습하는 것이므로 while문 안에서 들어가면 안되고 에피소드를 진행하는 for문에 안에 있어야합니다.

3. while문이 끝나지 않는 문제가 있었습니다. action을 버퍼에 담아서 확인해본 결과, agent가 목적지를 찾기 위해 action을 진행하면서 계속 같은 자리에서 머무는 경우가 있었습니다. 그래서 이 문제를 해결하기위해서 step제한을 넣어줬습니다.

4. 버퍼에 충분한 데이터가 담기지 않아서 랜덤샘플시 데이터가 샘플로 추출할 개수보다 부족하다는 오류가 떠서 continue를 사용해 버퍼에 데이터가 샘플링할 데이터보다 작을 경우 건너뛰도록 했습니다.

5. error code에서 1번 문제 때문에 학습부분이 실행되지 않아서 오류로 뜨지않았는데 loss 계산부분에서 W1, W2 값 두개를 바꿔주는데 loss식 안에 W2밖에 존재하지 않아서 오류가 떴습니다. 이 문제를 해결하기위해 네트워크를 통해 예측값을 구하는 부분을 한 줄로 풀어써줬습니다.

6. 샘플링을 통해 학습할 때마다 optimizer를 사용했을 때 학습시간이 너무 길어서 DQN 강의 1을 보고 np.vstack을 사용한 결과 좀 더 빠르게 학습을 할 수 있었습니다. 강의에서는 state를 넣어 줬는데 state는 숫자 하나이고 x_stack은 [0,16] 벡터라 결합이 되지 않았습니다. 그래서 원핫인코딩을 사용해서 해결했습니다.

7. cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:MatMul] 오류가 발생했습니다. np.empty를 생성할 때 dtype을 맞춰주지않아서 생긴 오류입니다. 



### DQN 과정

1. 네트워크를 만들고 안에 있는 가중치값들을 초기화 합니다..
2. 환경 만들기
3. 루프 시작

  - E-greedy방법을 통해 action을 랜덤하게 선택하거나, state를 넣어 네트워크를 통해 action을 받아옵니다.
  - env.step을 통해 action을 취해서 얻는 state_next, reward, done 정보와 state, action 을 바로 학습하지않고 버퍼에 저장합니다.
  - 루프에 10번에 1번마다 버퍼에서 랜덤하게 데이터를 샘플해서 네트워크를 학습합니다. (현재 코드)
